In [1]:
#데이터 불러오기
import pandas as pd
wine = pd.read_csv('data/wine.csv')

In [2]:
#clss 열 타겟, 나머지 열을 툭성 배열애 저장한다
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
#훈련세트와 테스트세트를 나눈다
from sklearn.model_selection import train_test_split
 
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [12]:
sub_input, val_input, sub_target, val_target = train_test_split(
        train_input, train_target, test_size=0.2, random_state = 42
)

In [13]:
sub_input.shape, val_input.shape

((4157, 3), (1040, 3))

In [14]:
#모델 훈련 및 평가
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


#### 교차검증

In [15]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target) #결정트리 + train_input + train_target
print(scores)

{'fit_time': array([0.01001143, 0.00697732, 0.00797582, 0.00897431, 0.00699735]), 'score_time': array([0.00096369, 0.        , 0.00099802, 0.        , 0.0009954 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [16]:
import numpy as np

np.mean(scores['test_score'])

0.855300214703487

In [17]:
from sklearn.model_selection import StratifiedKFold
 
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [18]:
#10개-폴드 교차 검증 수행
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42) #10개로 나눠 넣어주고 섞겠다
scores = cross_validate(dt, train_input, train_target, cv = splitter)

np.mean(scores['test_score'])

0.8574181117533719

#### 하이퍼 파라미터 튜닝

In [19]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [20]:
#그리드서치 객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [21]:
#훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [22]:
dt = gs.best_estimator_ #결정 트리 모델을 갖고옴
print(dt.score(train_input, train_target))

0.9615162593804117


In [23]:
#매개변수 중 가장 좋은 값 확인(최적의 값)
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [24]:
#5번 교차 검증으로 얻은 점수 출력
print(gs.cv_results_['mean_test_score']) #mean_test_score : 검증 값

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [25]:
#최고값 가져오기
best_index = np.argmax(gs.cv_results_['mean_test_score']) #가장 높은 값을 argmax()로 뽑아냄
print(gs.cv_results_['params'][best_index]) #0번쨰 해당하는 데이터(가장 좋은 매개변수) 가져옴

{'min_impurity_decrease': 0.0001}


In [26]:
#조금 더 복잡한 매개변수 조합 탐색
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
         } #매개변수 3개

In [27]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1) # n_jobs = -1 : 코어를 다 사용하겠다
gs.fit(train_input, train_target) #학습

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [28]:
#최상의 매개변수 조합 확인
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [30]:
#검증점수
gs.cv_results_['mean_test_score']

array([0.85780355, 0.85799604, 0.85799604, ..., 0.86126601, 0.86165063,
       0.86357629])

In [31]:
#최상의 교차검증점수 확인
np.max(gs.cv_results_['mean_test_score']) #mean_test_score : 검증점수, max : 검증점수 중 가장 큰 값

0.8683865773302731

In [32]:
#랜덤 서치
from scipy.stats import uniform, randint

rgen = randint(0, 10) #0~9까지 10개의 랜덤값을 뽑아내라, 골고루 샘플링하겠다
rgen.rvs(10)

array([4, 9, 7, 8, 5, 2, 8, 5, 7, 8])

In [33]:
#1000개 샘플링
np.unique(rgen.rvs(1000), return_counts=True) #.unique : 중복되지 않은 데이터만 추출

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 103, 100, 115,  94, 103,  90, 101, 108,  88], dtype=int64))

In [34]:
ugen = uniform(0, 1) #uniform : 실수값
ugen.rvs(10) #10개의 실수값을 뽑아내겠다

array([0.63704548, 0.95331706, 0.25497619, 0.57636433, 0.63056057,
       0.23901799, 0.44777221, 0.31278033, 0.8833524 , 0.86820959])

In [35]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [36]:
#랜덤서치 시작
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                            n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000206D3440308>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000206D57B49C8>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000206D3E02248>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000206D77833C8>},
                   random_state=42)

In [37]:
#최적의 매개변수 조합
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [38]:
#검증점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [39]:
#테스트 성능 확인
dt = gs.best_estimator_
 
print(dt.score(test_input, test_target))

0.86
